
## Introduction
#### Background
Multifamily is becoming an established sector of the UK property market. From less than 1% in 2014, it now accounts for 7% of total UK real estate investment. 2020 will likely see new entrants to the market, and current investors will continue to build their portfolios. As a result, we expect total multifamily investment in 2020 to significantly exceed 2019. ("From the UK Real Estate Outlook 2020")

#### Business Problem
In this scenario, it is urgent to adopt machine learning tools in order to assist homebuyers clientele in London to make wise and effective decisions. As a result, the business problem we are currently posing is: how could we provide support to homebuyers clientele in to purchase a suitable real estate in London in this uncertain economic and financial scenario?

To solve this business problem, we are going to cluster London neighborhoods in order to recommend venues and the current average price of real estate where homebuyers can make a real estate investment. We will recommend profitable venues according to amenities and essential facilities surrounding such venues i.e. elementary schools, high schools, hospitals & grocery stores.

## Data
Data on London properties and the relative price paid data were extracted from the HM Land Registry (http://landregistry.data.gov.uk/). The following fields comprise the address data included in Price Paid Data: Postcode; PAON Primary Addressable Object Name. Typically the house number or name; SAON Secondary Addressable Object Name. If there is a sub-building, for example, the building is divided into flats, there will be a SAON; Street; Locality; Town/City; District; County.

To explore and target recommended locations across different venues according to the presence of amenities and essential facilities, we will access data through FourSquare API interface and arrange them as a dataframe for visualization. By merging data on London properties and the relative price paid data from the HM Land Registry and data on amenities and essential facilities surrounding such properties from FourSquare API interface, we will be able to recommend profitable real estate investments.

## Methodology

The Methodology section will describe the main components of our data analysis, modelling and prediction system. The Methodology section comprises of four stages:

1. Collection and Inspection Data
2. Exploration and Understanding of the Data
3. Preparation and preprocessing of the Data
4. Modeling of the Data

#### 1. Collection and Inspection of Data

In [1]:
# importing the required libraries
import os # Operating System
import numpy as np
import pandas as pd
import datetime as dt # Datetime
import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

import folium # map rendering library

import hmac

from geopy.distance import geodesic

# import k-means from clustering stage
from sklearn.cluster import KMeans

print('Libraries imported.')

Libraries imported.


In [2]:
# Read the data from http://landregistry.data.gov.uk/
df = pd.read_csv("http://prod2.publicdata.landregistry.gov.uk.s3-website-eu-west-1.amazonaws.com/pp-2018.csv")

In [3]:
print("Current Features: ", df.columns)

Current Features:  Index(['{79A74E22-41E2-1289-E053-6B04A8C01627}', '60000', '2018-06-29 00:00',
       'DH3 1DN', 'F', 'N', 'L', '20', 'Unnamed: 8', 'BEACONSFIELD TERRACE',
       'BIRTLEY', 'CHESTER LE STREET', 'GATESHEAD', 'TYNE AND WEAR', 'B', 'A'],
      dtype='object')


#### 2. Exploration and Data understanding

In [4]:
# we display the first five rows of the data as follows:
df.head()

,{79A74E22-41E2-1289-E053-6B04A8C01627},60000,2018-06-29 00:00,DH3 1DN,F,N,L,20,Unnamed: 8,BEACONSFIELD TERRACE,BIRTLEY,CHESTER LE STREET,GATESHEAD,TYNE AND WEAR,B,A
0,{79A74E22-41E3-1289-E053-6B04A8C01627},149950,2018-06-14 00:00,DH4 6NZ,T,Y,F,50,NaN,GLANVILLE DRIVE,NaN,HOUGHTON LE SPRING,SUNDERLAND,TYNE AND WEAR,A,A
1,{79A74E22-41E4-1289-E053-6B04A8C01627},164950,2018-06-29 00:00,SR2 0FD,S,Y,F,6,NaN,WILSHIRE CLOSE,NaN,SUNDERLAND,SUNDERLAND,TYNE AND WEAR,A,A
2,{79A74E22-41E5-1289-E053-6B04A8C01627},224950,2018-06-29 00:00,SR2 0FA,D,Y,F,47,NaN,WOODHAM DRIVE,NaN,SUNDERLAND,SUNDERLAND,TYNE AND WEAR,A,A
3,{79A74E22-41E6-1289-E053-6B04A8C01627},129950,2018-06-28 00:00,DH4 6NY,S,Y,F,65A,NaN,CHALK HILL ROAD,NaN,HOUGHTON LE SPRING,SUNDERLAND,TYNE AND WEAR,A,A
4,{79A74E22-41E7-1289-E053-6B04A8C01627},144395,2018-02-23 00:00,NE31 2EL,T,Y,F,9,NaN,TURNBERRY DRIVE,NaN,HEBBURN,SOUTH TYNESIDE,TYNE AND WEAR,A,A


In [5]:
print("The number of rows are: ", df.shape[0])
print("The number of columns are: ", df.shape[1])

The number of rows are:  1031509
The number of columns are:  16


#### 3. Data preparation and processing

At this stage, we prepare our dataset for the modelling process, opting for the most suitable machine learning algorithm for our scope.<br>
Accordingly, we perform the following steps:

  - Rename the column names
  - Format the date column
  - Sort data by date of sale
  - Select data only for the city of London
  - Make a list of street names in London
  - Calculate the street-wise average price of the property
  - Read the street-wise coordinates into a data frame, eliminating recurring word London from individual names
  - Join the data to find the coordinates of locations which fit into client's budget
  - Plot recommended locations on London map along with current market prices

In [6]:
# Assign meaningful column names
df.columns = ['TUID', 'Price', 'Date_Transfer', 'Postcode', 'Prop_Type', 'Old_New', 'Duration', 'PAON', \
              'SAON', 'Street', 'Locality', 'Town_City', 'District', 'County', 'PPD_Cat_Type', 'Record_Status']

In [7]:
# Format the date column
df['Date_Transfer'] = df['Date_Transfer'].apply(pd.to_datetime)

# Delete all obsolete transactions which were done before 2016
df.drop(df[df.Date_Transfer.dt.year < 2016].index, inplace=True)

# Sort by Date of Sale
df.sort_values(by=['Date_Transfer'],ascending=[False],inplace=True)

In [8]:
df_london = df.query("Town_City == 'LONDON'")

# Make a list of street names in LONDON
streets = df_london['Street'].unique().tolist()

In [9]:
df_grp_price = df_london.groupby(['Street'])['Price'].mean().reset_index()

# Give meaningful names to the columns
df_grp_price.columns = ['Street', 'Avg_Price']

In [10]:
#Input your upper and lower budget limits - Find the locations of df_grp_price which fits your budget
df_affordable = df_grp_price.query("(Avg_Price >= 2200000) & (Avg_Price <= 2500000)")

In [11]:
# Display the dataframe
df_affordable

,Street,Avg_Price
196,ALBION SQUARE,2450000.0
390,ANHALT ROAD,2435000.0
405,ANSDELL TERRACE,2250000.0
422,APPLEGARTH ROAD,2400000.0
857,BARONSMEAD ROAD,2375000.0
...,...,...
13733,WILFRED STREET,2410538.5
13759,WILLOW BRIDGE ROAD,2425000.0
13779,WILSON STREET,2257500.0
13808,WINCHENDON ROAD,2350000.0


In [12]:
for index, item in df_affordable.iterrows():
    print(f"index: {index}")
    print(f"item: {item}")
    print(f"item.Street only: {item.Street}")

index: 196
item: Street       ALBION SQUARE
Avg_Price         2.45e+06
Name: 196, dtype: object
item.Street only: ALBION SQUARE
index: 390
item: Street       ANHALT ROAD
Avg_Price      2.435e+06
Name: 390, dtype: object
item.Street only: ANHALT ROAD
index: 405
item: Street       ANSDELL TERRACE
Avg_Price           2.25e+06
Name: 405, dtype: object
item.Street only: ANSDELL TERRACE
index: 422
item: Street       APPLEGARTH ROAD
Avg_Price            2.4e+06
Name: 422, dtype: object
item.Street only: APPLEGARTH ROAD
index: 857
item: Street       BARONSMEAD ROAD
Avg_Price          2.375e+06
Name: 857, dtype: object
item.Street only: BARONSMEAD ROAD
index: 983
item: Street       BEAUCLERC ROAD
Avg_Price          2.48e+06
Name: 983, dtype: object
item.Street only: BEAUCLERC ROAD
index: 1105
item: Street       BELVEDERE DRIVE
Avg_Price           2.34e+06
Name: 1105, dtype: object
item.Street only: BELVEDERE DRIVE
index: 1218
item: Street       BICKENHALL STREET
Avg_Price           2.2085e+06
N

In [13]:
# let's get the geographical coordinates of London.
address = 'London, UK'

geolocator = Nominatim(user_agent="ln_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of London City are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of London City are 51.5073219, -0.1276474.


In [14]:
df_affordable['city_coord'] = df_affordable['Street'].apply(geolocator.geocode).apply(lambda x: (x.latitude, x.longitude))

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [15]:
# In the above code, we've created the coordinates of the streets
df_affordable.head()

,Street,Avg_Price,city_coord
196,ALBION SQUARE,2450000.0,"(-41.27375755, 173.28939323910353)"
390,ANHALT ROAD,2435000.0,"(29.7127696, -98.0948057)"
405,ANSDELL TERRACE,2250000.0,"(51.5000051, -0.1891537)"
422,APPLEGARTH ROAD,2400000.0,"(53.749244, -0.32678)"
857,BARONSMEAD ROAD,2375000.0,"(51.4773147, -0.239457)"


In [16]:
# We now split those coordinates into two columns
df_affordable[['Latitude', 'Longitude']] = df_affordable['city_coord'].apply(pd.Series)

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/pandas/core/frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [17]:
df_affordable.head()

,Street,Avg_Price,city_coord,Latitude,Longitude
196,ALBION SQUARE,2450000.0,"(-41.27375755, 173.28939323910353)",-41.273758,173.289393
390,ANHALT ROAD,2435000.0,"(29.7127696, -98.0948057)",29.712770,-98.094806
405,ANSDELL TERRACE,2250000.0,"(51.5000051, -0.1891537)",51.500005,-0.189154
422,APPLEGARTH ROAD,2400000.0,"(53.749244, -0.32678)",53.749244,-0.326780
857,BARONSMEAD ROAD,2375000.0,"(51.4773147, -0.239457)",51.477315,-0.239457


In [18]:
print("New number of rows is: ", df_affordable.shape[0])

New number of rows is:  159


In [19]:
# the city_coord column is now redundant
df = df_affordable.drop(columns=['city_coord'])

In [20]:
df.head()

,Street,Avg_Price,Latitude,Longitude
196,ALBION SQUARE,2450000.0,-41.273758,173.289393
390,ANHALT ROAD,2435000.0,29.712770,-98.094806
405,ANSDELL TERRACE,2250000.0,51.500005,-0.189154
422,APPLEGARTH ROAD,2400000.0,53.749244,-0.326780
857,BARONSMEAD ROAD,2375000.0,51.477315,-0.239457


In [21]:
# create map of London using latitude and longitude values
map_london = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, price, street in zip(df['Latitude'], df['Longitude'], df['Avg_Price'], df['Street']):
    label = '{}, {}'.format(street, price)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_london)  
    
map_london

In [22]:
#Define Foursquare Credentials and Version
CLIENT_ID = 'TTDNQ2SHABH0G0EDPL3CG2AGWNOUAVCWMJQQRWWRBA5Z1VDB'
CLIENT_SECRET = 'JRAFRKNBAAW5CJ1FUV2KVFOZMDYXUIYFKSAT423BJCZ1OAPT'
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: TTDNQ2SHABH0G0EDPL3CG2AGWNOUAVCWMJQQRWWRBA5Z1VDB
CLIENT_SECRET:JRAFRKNBAAW5CJ1FUV2KVFOZMDYXUIYFKSAT423BJCZ1OAPT


We can now proceed to the Modeling phase. We will analyze neighborhoods to recommend real estates where home buyers can make a real estate investment. We will then recommend profitable venues according to amenities and essential facilities surrounding such venues i.e. elementary schools, high schools, hospitals & grocery stores.

#### 4. Modeling

Since we are done with exploring the dataset and gaining insights from it, we will now begin the modelling phase. We will use a clustering algorithm to cluster the real estate locations. Specifically, we will use the k-means clustering algorithm as it is fast and efficient in terms of computational cost and it is also highly flexible to account for uncertainties in the real estate market of London.

In [23]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Street', 
                  'Street Latitude', 
                  'Street Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [24]:
# Run the above function on each location and create a new dataframe called location_venues and display it.
location_venues = getNearbyVenues(names=df['Street'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

ALBION SQUARE
ANHALT ROAD
ANSDELL TERRACE
APPLEGARTH ROAD
BARONSMEAD ROAD
BEAUCLERC ROAD
BELVEDERE DRIVE
BICKENHALL STREET
BIRCHLANDS AVENUE
BRAMPTON GROVE
BRIARDALE GARDENS
BROOKWAY
BURY WALK
CALLCOTT STREET
CAMPDEN HILL ROAD
CAMPION ROAD
CANNING PLACE
CARLISLE ROAD
CARLTON GARDENS
CARLYLE COURT
CHALCOT SQUARE
CHARLES LANE
CHELSEA CRESCENT
CHESTER CLOSE NORTH
CHEYNE COURT
CHEYNE ROW
CHISWICK MALL
CITY ROAD
CLARENDON STREET
CLONCURRY STREET
COLBECK MEWS
COLLEGE CRESCENT
CORNWALL TERRACE MEWS
COURT LANE GARDENS
CRESCENT GROVE
DALEBURY ROAD
DEWHURST ROAD
DORIA ROAD
DOWNSHIRE HILL
DUCHESS WALK
ECCLESTON SQUARE MEWS
EGBERT STREET
EGERTON PLACE
ELM PARK ROAD
FLORAL STREET
FRANK DIXON WAY
FULTON MEWS
GERARD ROAD
GERRARD ROAD
GIRDLERS ROAD
GLOUCESTER CRESCENT
GORDON PLACE
GRAFTON SQUARE
GRAHAM TERRACE
HARMAN DRIVE
HARRIS STREET
HAVANNAH STREET
HAZLEWELL ROAD
HEREFORD MEWS
HERONDALE AVENUE
HIGHGATE HIGH STREET
HIGHWOOD HILL
HILLGATE PLACE
HOLLYCROFT AVENUE
HOLLYWOOD MEWS
HONEYWELL ROAD
HORTENS

In [25]:
location_venues.head()

,Street,Street Latitude,Street Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,ALBION SQUARE,-41.273758,173.289393,The Free House,-41.273340,173.287364,Bar
1,ALBION SQUARE,-41.273758,173.289393,Queen's Gardens,-41.273671,173.291383,Park
2,ALBION SQUARE,-41.273758,173.289393,The Indian Cafe,-41.273308,173.286530,Indian Restaurant
3,ALBION SQUARE,-41.273758,173.289393,Urban,-41.274355,173.286317,New American Restaurant
4,ALBION SQUARE,-41.273758,173.289393,Fish Stop,-41.276010,173.289592,Fish & Chips Shop


In [26]:
print("The number of rows in this dataframe are: ", location_venues.shape[0])

The number of rows in this dataframe are:  4518


In [27]:
location_venues.groupby('Street').count()

,Street Latitude,Street Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Street,,,,,,
ALBION SQUARE,27,27,27,27,27,27
ANHALT ROAD,6,6,6,6,6,6
ANSDELL TERRACE,73,73,73,73,73,73
APPLEGARTH ROAD,5,5,5,5,5,5
BARONSMEAD ROAD,16,16,16,16,16,16
...,...,...,...,...,...,...
WESTMORELAND PLACE,16,16,16,16,16,16
WHITFIELD STREET,16,16,16,16,16,16
WILFRED STREET,13,13,13,13,13,13


In [28]:
# get the List of Unique Categories
print('There are {} uniques categories.'.format(len(location_venues['Venue Category'].unique())))

There are 344 uniques categories.


In [29]:
# one hot encoding
venues_onehot = pd.get_dummies(location_venues[['Venue Category']], prefix="", prefix_sep="")

# add street column back to dataframe
venues_onehot['Street'] = location_venues['Street'] 

# move street column to the first column
fixed_columns = [venues_onehot.columns[-1]] + list(venues_onehot.columns[:-1])

#fixed_columns
venues_onehot = venues_onehot[fixed_columns]

venues_onehot.head()

,Street,ATM,Acai House,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,...,Vietnamese Restaurant,Warehouse Store,Waterfront,Weight Loss Center,Windmill,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo
0,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
london_grouped = venues_onehot.groupby('Street').mean().reset_index()
london_grouped.head()

,Street,ATM,Acai House,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,...,Vietnamese Restaurant,Warehouse Store,Waterfront,Weight Loss Center,Windmill,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo
0,ALBION SQUARE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1,ANHALT ROAD,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,ANSDELL TERRACE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.013699,0.0,0.0,0.0,0.0
3,APPLEGARTH ROAD,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
4,BARONSMEAD ROAD,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


In [31]:
london_grouped.shape

(141, 345)

In [32]:
# What are the top 5 venues/facilities nearby profitable real estate investments?#

num_top_venues = 5

for hood in london_grouped['Street']:
    print("----"+hood+"----")
    temp = london_grouped[london_grouped['Street'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----ALBION SQUARE----
               venue  freq
0               Café  0.22
1                Pub  0.07
2  Indian Restaurant  0.07
3                Bar  0.07
4        Coffee Shop  0.07


----ANHALT ROAD----
          venue  freq
0  Intersection  0.17
1  Dance Studio  0.17
2   Coffee Shop  0.17
3         Hotel  0.17
4           Gym  0.17


----ANSDELL TERRACE----
                venue  freq
0               Hotel  0.08
1          Restaurant  0.05
2           Juice Bar  0.05
3                 Pub  0.05
4  Italian Restaurant  0.05


----APPLEGARTH ROAD----
             venue  freq
0              Bar   0.2
1        Nightclub   0.2
2   Sandwich Place   0.2
3  Auto Dealership   0.2
4           Casino   0.2


----BARONSMEAD ROAD----
                 venue  freq
0    Food & Drink Shop  0.12
1  Indie Movie Theater  0.06
2          Pizza Place  0.06
3                 Park  0.06
4      Nature Preserve  0.06


----BEAUCLERC ROAD----
                venue  freq
0         Coffee Shop  0.12
1          

In [33]:
# Define a function to return the most common venues/facilities nearby real estate investments#

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [34]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Street']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

In [35]:
# create a new dataframe
venues_sorted = pd.DataFrame(columns=columns)
venues_sorted['Street'] = london_grouped['Street']

for ind in np.arange(london_grouped.shape[0]):
    venues_sorted.iloc[ind, 1:] = return_most_common_venues(london_grouped.iloc[ind, :], num_top_venues)

In [36]:
venues_sorted.head()

,Street,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,ALBION SQUARE,Café,Indian Restaurant,Pub,Restaurant,Coffee Shop,Bar,Beer Garden,Paper / Office Supplies Store,Fish & Chips Shop,Supermarket
1,ANHALT ROAD,Movie Theater,Coffee Shop,Hotel,Intersection,Gym,Dance Studio,English Restaurant,Escape Room,Ethiopian Restaurant,Event Space
2,ANSDELL TERRACE,Hotel,Indian Restaurant,Café,Pub,Juice Bar,Italian Restaurant,Restaurant,Clothing Store,French Restaurant,Sporting Goods Shop
3,APPLEGARTH ROAD,Sandwich Place,Nightclub,Auto Dealership,Casino,Bar,Flea Market,Fish Market,English Restaurant,Escape Room,Ethiopian Restaurant
4,BARONSMEAD ROAD,Food & Drink Shop,Breakfast Spot,Nature Preserve,Pizza Place,Movie Theater,Community Center,Indie Movie Theater,Pub,Thai Restaurant,Restaurant


In [37]:
venues_sorted.shape

(141, 11)

In [38]:
london_grouped.shape

(141, 345)

In [39]:
london_grouped=df

After our inspection of venues/facilities/amenities situated near the most profitable real estate investments in London, we shall begin by clustering properties by venues/facilities/amenities nearby.

In [40]:
# set number of clusters
kclusters = 5

london_grouped_clustering = london_grouped.drop('Street', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(london_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:50]

array([2, 0, 1, 0, 3, 2, 3, 1, 1, 2, 0, 0, 2, 3, 3, 2, 0, 1, 2, 4, 4, 0,
       2, 2, 1, 0, 4, 2, 1, 0, 3, 0, 3, 3, 4, 0, 0, 3, 1, 2, 3, 4, 1, 4,
       1, 1, 4, 1, 1, 0], dtype=int32)

In [41]:
#Dataframe to include Clusters

london_grouped_clustering=df
london_grouped_clustering.head()

,Street,Avg_Price,Latitude,Longitude
196,ALBION SQUARE,2450000.0,-41.273758,173.289393
390,ANHALT ROAD,2435000.0,29.712770,-98.094806
405,ANSDELL TERRACE,2250000.0,51.500005,-0.189154
422,APPLEGARTH ROAD,2400000.0,53.749244,-0.326780
857,BARONSMEAD ROAD,2375000.0,51.477315,-0.239457


In [42]:
london_grouped_clustering.shape

(159, 4)

In [43]:
df.shape

(159, 4)

In [44]:
london_grouped_clustering.dtypes

Street        object
Avg_Price    float64
Latitude     float64
Longitude    float64
dtype: object

In [45]:
df.dtypes

Street        object
Avg_Price    float64
Latitude     float64
Longitude    float64
dtype: object

In [46]:
# add clustering labels
london_grouped_clustering['Cluster Labels'] = kmeans.labels_

# merge london_grouped with london_data to add latitude/longitude for each neighborhood
london_grouped_clustering = london_grouped_clustering.join(venues_sorted.set_index('Street'), on='Street')

london_grouped_clustering.head(30) # check the last columns!

,Street,Avg_Price,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
196,ALBION SQUARE,2.450000e+06,-41.273758,173.289393,2,Café,Indian Restaurant,Pub,Restaurant,Coffee Shop,Bar,Beer Garden,Paper / Office Supplies Store,Fish & Chips Shop,Supermarket
390,ANHALT ROAD,2.435000e+06,29.712770,-98.094806,0,Movie Theater,Coffee Shop,Hotel,Intersection,Gym,Dance Studio,English Restaurant,Escape Room,Ethiopian Restaurant,Event Space
405,ANSDELL TERRACE,2.250000e+06,51.500005,-0.189154,1,Hotel,Indian Restaurant,Café,Pub,Juice Bar,Italian Restaurant,Restaurant,Clothing Store,French Restaurant,Sporting Goods Shop
422,APPLEGARTH ROAD,2.400000e+06,53.749244,-0.326780,0,Sandwich Place,Nightclub,Auto Dealership,Casino,Bar,Flea Market,Fish Market,English Restaurant,Escape Room,Ethiopian Restaurant
857,BARONSMEAD ROAD,2.375000e+06,51.477315,-0.239457,3,Food & Drink Shop,Breakfast Spot,Nature Preserve,Pizza Place,Movie Theater,Community Center,Indie Movie Theater,Pub,Thai Restaurant,Restaurant
983,BEAUCLERC ROAD,2.480000e+06,51.499577,-0.229033,2,Pub,Coffee Shop,Hotel,Bed & Breakfast,Chinese Restaurant,Grocery Store,Thai Restaurant,Falafel Restaurant,Cocktail Bar,Ice Cream Shop
1105,BELVEDERE DRIVE,2.340000e+06,38.072439,-78.459970,3,Pool,Playground,Athletics & Sports,Zoo,Farm,English Restaurant,Escape Room,Ethiopian Restaurant,Event Space,Exhibit
1218,BICKENHALL STREET,2.208500e+06,51.521201,-0.158908,1,Coffee Shop,Italian Restaurant,Pizza Place,Chinese Restaurant,Gastropub,Hotel,Café,Sandwich Place,Pub,Garden
1256,BIRCHLANDS AVENUE,2.217000e+06,51.448394,-0.160468,1,Pub,Bakery,Coffee Shop,French Restaurant,Chinese Restaurant,Lake,Train Station,Brewery,Pizza Place,Dance Studio
1556,BRAMPTON GROVE,2.456875e+06,51.587314,-0.225575,2,Coffee Shop,Grocery Store,Restaurant,Bagel Shop,Sushi Restaurant,Gym / Fitness Center,Korean Restaurant,Fried Chicken Joint,Noodle House,Turkish Restaurant


In [47]:
# Create Map

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(london_grouped_clustering['Latitude'], london_grouped_clustering['Longitude'], london_grouped_clustering['Street'], london_grouped_clustering['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### the cluster are shown below

In [48]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 0, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
390,2435000.0,Movie Theater,Coffee Shop,Hotel,Intersection,Gym,Dance Studio,English Restaurant,Escape Room,Ethiopian Restaurant,Event Space
422,2400000.0,Sandwich Place,Nightclub,Auto Dealership,Casino,Bar,Flea Market,Fish Market,English Restaurant,Escape Room,Ethiopian Restaurant
1635,2397132.0,Breakfast Spot,Grocery Store,Coffee Shop,Park,Seafood Restaurant,Convenience Store,Hookah Bar,Hotel Bar,Egyptian Restaurant,Electronics Store
1800,2400000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2162,2425000.0,Clothing Store,American Restaurant,Gym / Fitness Center,Gym,Electronics Store,Stationery Store,Department Store,Basketball Court,Music Venue,Chinese Restaurant


In [49]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 1, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
405,2250000.0,Hotel,Indian Restaurant,Café,Pub,Juice Bar,Italian Restaurant,Restaurant,Clothing Store,French Restaurant,Sporting Goods Shop
1218,2208500.0,Coffee Shop,Italian Restaurant,Pizza Place,Chinese Restaurant,Gastropub,Hotel,Café,Sandwich Place,Pub,Garden
1256,2217000.0,Pub,Bakery,Coffee Shop,French Restaurant,Chinese Restaurant,Lake,Train Station,Brewery,Pizza Place,Dance Studio
2229,2200000.0,Asian Restaurant,Baseball Field,Mexican Restaurant,Sandwich Place,Chinese Restaurant,Sushi Restaurant,Pharmacy,Spa,Fish Market,Filipino Restaurant
2642,2250000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 2, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
196,2450000.0,Café,Indian Restaurant,Pub,Restaurant,Coffee Shop,Bar,Beer Garden,Paper / Office Supplies Store,Fish & Chips Shop,Supermarket
983,2480000.0,Pub,Coffee Shop,Hotel,Bed & Breakfast,Chinese Restaurant,Grocery Store,Thai Restaurant,Falafel Restaurant,Cocktail Bar,Ice Cream Shop
1556,2456875.0,Coffee Shop,Grocery Store,Restaurant,Bagel Shop,Sushi Restaurant,Gym / Fitness Center,Korean Restaurant,Fried Chicken Joint,Noodle House,Turkish Restaurant
1983,2492500.0,English Restaurant,Supermarket,Hardware Store,Café,Discount Store,Gym,Rental Car Location,Fast Food Restaurant,American Restaurant,Park
2139,2461000.0,Windmill,Soccer Stadium,Bus Station,Zoo,Farm,English Restaurant,Escape Room,Ethiopian Restaurant,Event Space,Exhibit


In [51]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 3, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
857,2375000.0,Food & Drink Shop,Breakfast Spot,Nature Preserve,Pizza Place,Movie Theater,Community Center,Indie Movie Theater,Pub,Thai Restaurant,Restaurant
1105,2340000.0,Pool,Playground,Athletics & Sports,Zoo,Farm,English Restaurant,Escape Room,Ethiopian Restaurant,Event Space,Exhibit
2071,2375000.0,Pub,Park,Indian Restaurant,Hotel,Yoga Studio,Pizza Place,Coffee Shop,Persian Restaurant,Portuguese Restaurant,Record Shop
2132,2379652.7,Café,Pub,Restaurant,Clothing Store,Park,Greek Restaurant,Garden,Thai Restaurant,Bakery,English Restaurant
2948,2367500.0,Hotel,Pub,Garden,Coffee Shop,Chinese Restaurant,Café,Mediterranean Restaurant,Bar,Cocktail Bar,Italian Restaurant


In [52]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 4, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2246,2.300000e+06,Farm,Zoo,Farmers Market,English Restaurant,Escape Room,Ethiopian Restaurant,Event Space,Exhibit,Factory,Falafel Restaurant
2410,2.286679e+06,Café,Italian Restaurant,Pub,Coffee Shop,Bar,French Restaurant,Park,Convenience Store,Caribbean Restaurant,Beer Store
2690,2.287500e+06,Pub,Reservoir,Brewery,Gift Shop,Art Museum,Playground,Flea Market,Factory,Food,Electronics Store
3381,2.298000e+06,Hotel,Zoo,Farm,Electronics Store,English Restaurant,Escape Room,Ethiopian Restaurant,Event Space,Exhibit,Factory
4289,2.265000e+06,Pub,Farm,Electronics Store,English Restaurant,Escape Room,Ethiopian Restaurant,Event Space,Exhibit,Factory,Falafel Restaurant


We see that although West London (Notting Hill, Kensington, Chelsea, Marylebone) and North-West London (Hampsted) might be considered highly profitable venues to purchase real estate according to amenities and essential facilities surrounding such venues i.e. elementary schools, high schools, hospitals & grocery stores, South-West London (Wandsworth, Balham) and North-West London (Isliington) are rising as next future elite venues with a wide range of amenities and facilities. Accordingly, one might target under-priced real estates in these areas of London in order to make a profit in the near future.

We have found two main patterns. The first pattern refers to Clusters 0, 2 and 4; here we may target home buyers prone to live in 'green' areas with parks, waterfronts. The second pattern refers to Clusters 1 and 3; here we may target individuals who love pubs, theatres and soccer.